# Setup inicial.

In [1]:
%pip install -q --upgrade transformers peft trl bitsandbytes accelerate datasets pandas
%pip install -q --upgrade gdown huggingface_hub ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
qa_file_path = './datasets_lora_nlp2025/qa_dataset.csv'
model_id = 'meta-llama/Llama-3.2-1B-Instruct'
output_model_dir = "./lora_folder"

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import torch
import os

In [4]:
from google.colab import output
output.enable_custom_widget_manager()

ModuleNotFoundError: No module named 'google'

In [ ]:
import torch
print(torch.cuda.is_available()) # Should be True if CUDA is detected
print(torch.version.cuda)

True
12.4


# Importando Datasets

Pega todas as noticias do site https://www.cnnbrasil.com.br referentes à Champions League 24/25.

In [ ]:
import gdown

url = "https://drive.google.com/drive/folders/10NbeNi3B_D448Fxst67bJQ7-s1GXx4nZ?usp=drive_link"
gdown.download_folder(url, quiet=False, use_cookies=False)


# Pegando as noticias.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re

BASE_URL = 'https://www.cnnbrasil.com.br/esportes/futebol/futebol-internacional/champions-league/pagina/'
HEADERS = {'User-Agent': 'Mozilla/5.0'}
OUTPUT_FILE = 'cnn_champions_news.txt'

def get_news_links_cnn(page_number):
    url = f'{BASE_URL}{page_number}/'
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f'Erro ao acessar a página {page_number}: {e}')
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if re.match(r'^https://www\.cnnbrasil\.com\.br/esportes/futebol/futebol-internacional/champions-league/.+', href) and "onde-assistir" not in href:
            links.append(href)
    return list(set(links))  # remover duplicatas

def get_article_text_cnn(url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f'Erro ao acessar a notícia {url}: {e}')
        return ''

    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p', class_="my-5 break-words")
    text = '\n'.join(p.get_text(strip=True) for p in paragraphs)
    return text

def get_news_cnn():
    all_links = set()
    for page in range(1, 38):
        print(f'Processando página {page}...')
        links = get_news_links_cnn(page)
        print(f'Encontrados {len(links)} links na página {page}.')
        all_links.update(links)
        time.sleep(1)

    print(f'Total de links únicos encontrados: {len(all_links)}')

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        for idx, link in enumerate(all_links, 1):
            print(f'Lendo notícia {idx}/{len(all_links)}: {link}')
            article_text = get_article_text_cnn(link)
            if article_text:
                f.write(article_text + '\n\n')
            time.sleep(1)

    print(f'Finalizado. Notícias salvas em {OUTPUT_FILE}.')

get_news_cnn()


# Gerando dados sintéticos

## Configurando API do Google

In [ ]:
import os
from google.colab import userdata

try:
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    print("Chave de API do Gemini carregada dos Secrets.")
except KeyError:
    print("Erro: A chave 'GOOGLE_API_KEY' não foi encontrada nos Secrets.")

In [ ]:
import google.generativeai as genai  # Importa a biblioteca generativeai como genai
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

In [ ]:
models = genai.list_models()
for model in models:
    if('gemini' in model.name):
        print(model.name)

In [ ]:
def chamada_LLM(prompt,
                model_name='gemini-2.0-flash-001',
                # model_name='gemini-2.5-flash-preview-05-20',
                temperature=0.7,
                max_output_tokens=1000000):
  """
  Envia um prompt para um modelo Gemini e retorna a resposta gerada.

  Args:
    prompt: O prompt de texto a ser enviado ao modelo.
    model_name: O nome do modelo Gemini a ser utilizado (padrão: 'gemini-2.0-flash').
    temperature: Controla a aleatoriedade da saída (padrão: 0.7). Valores mais baixos tornam a saída mais determinística.
    max_output_tokens: O número máximo de tokens a serem gerados na resposta (padrão: 500).

  Returns:
    O texto da resposta gerada pelo modelo Gemini ou None em caso de erro.
  """
  try:
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": temperature,
            "max_output_tokens": max_output_tokens
        }
    )
    return response.text
  except Exception as e:
    print(f"Erro ao gerar resposta com o modelo {model_name}: {e}")
    return None



In [ ]:
# prompt simples tem os seguintes elementos
#### Instrução: uma tarefa ou instrução específica que se deseja que o modelo execute
#### Contexto: Informações externas ou contexto adicional que podem orientar o modelo a fornecer melhores respostas
#### Dados de entrada: entrada ou pergunta para a qual estamos interessados ​​​​em encontrar uma resposta
#### Indicador de saída: tipo ou formato da saída.
def prompt_simples(instrucao, contexto, indicador_saida):
  """
  Envia um prompt para o Gemini e retorna a resposta gerada.

  Args:
    instrucao: A tarefa ou instrução específica.
    contexto: Informações externas ou contexto adicional.
    dados_entrada: Entrada ou pergunta.
    indicador_saida: Tipo ou formato da saída desejado.

  Returns:
    A resposta gerada pelo Gemini ou None em caso de erro.
  """

  prompt = f"""

  Contexto: {contexto}

  Instrução: {instrucao}

  Indicador de saída: {indicador_saida}
  """

  print("Prompt length:", len(prompt), "chars.")
  return chamada_LLM(prompt)


instrucao = "Complete a frase da melhor maneira. frase: O céu é "
contexto = "Você é um físico"
indicador_saida = "Texto em portugues, com no máximo 10 palavras"

resposta = prompt_simples(instrucao, contexto, indicador_saida)

if resposta:
  print(resposta)

## Gerando instruções + resposta

### Função que envia o prompt.

- **Contexto**: arquivo e perguntas já feitas.
- **Instrução**: Gere X perguntas e respostas sobre o texto que está no contexto.
- **Descrição da saída**: Cada par pergunta/resposta deve estar no seguinte formato:id\n@Pergunta: descricao_pergunta@\n@Resposta: descricao_resposta@. Não enumere as perguntas. As perguntas devem ser relevantes para alguém que queira se informar sobre o assunto do contexto. As repostas tem que ser completamente baseada no texto do contexto. As respostas devem ser tão grandes quanto necessário para responder a pergunta satisfatoriamente. Não repita perguntas. No contexto também tem algumas perguntas já geradas anteriormente, não repita essas perguntas. As perguntas e respostas devem estar em português. Faça com que o conteudo do contexto inteiro seja abrangido pelas perguntas geradas. Deixa claro que a pergunta se refere a champions league 2024/2025 em alguma parte da pergunta ou da resposta. Lembre-se que algumas informações como número de pontos, colocação na tabela, etc., são dados que mudam com o tempo e o conteúdo do contexto ocorreu no passado, logo a data é importante nesses casos.

In [ ]:
def generate_intruction_response(file, questions, number):
    return prompt_simples(f"Gere {number} perguntas e respostas sobre o texto que está no contexto.",
                   file + questions,
                   "Cada par pergunta/resposta deve estar no seguinte formato:id\n@Pergunta: descricao_pergunta@\n@Resposta: descricao_resposta@. Não enumere as perguntas. As perguntas devem ser relevantes para alguém que queira se informar sobre o assunto do contexto. As repostas tem que ser completamente baseada no texto do contexto. As respostas devem ser tão grandes quanto necessário para responder a pergunta satisfatoriamente. Não repita perguntas. No contexto também tem algumas perguntas já geradas anteriormente, não repita essas perguntas. As perguntas e respostas devem estar em português. Faça com que o conteudo do contexto inteiro seja abrangido pelas perguntas geradas. Deixa claro que a pergunta se refere a champions league 2024/2025 em alguma parte da pergunta ou da resposta. Lembre-se que algumas informações como número de pontos, colocação na tabela, etc., são dados que mudam com o tempo e o conteúdo do contexto ocorreu no passado, logo a data é importante nesses casos.")

### Gerando perguntas para os textos base

Nessa parte o prompt acima é executado até o seguinte número de perguntas e respostas forem geradas para cada texto base.

- Artigos da wikipedia -> 450 perguntas e respostas
- Noticias da CNN -> 450 perguntas e respostas
- Relatorio do Gemini Deep Research -> 200 perguntas e repostas

In [ ]:
import re

sintetic_data = []
generated_questions = ""
datasets = [('wikipedia_articles_pt.txt', 450), ('cnn_champions_news.txt', 450), ('gemini_deep_research.txt', 100)]
for (file_name, quantity) in datasets:
    print(f"\n######\nGerando perguntas para o arquivo {file_name}...")
    print(f"Quantidade de perguntas a serem geradas: {quantity}")
    print("######\n")

    with open(f'datasets_lora_nlp2025/{file_name}', 'r', encoding='utf-8') as f:
        file_content = f.read()
        print(f"Conteúdo do arquivo {file_name} carregado na variável 'file_content'. Tamanho: {len(file_content)} caracteres.")

    parsed = []
    while len(parsed) < quantity:
        res = generate_intruction_response(file_content, generated_questions, min(100, quantity - len(parsed)))
        print("Caracteres na resposta:", len(res))

        blocks = res.split("@Pergunta:")[1:]
        for block in blocks:
            try:
                pergunta = block[:block.find("@")].strip('@\n ')

                begin_resposta = block.find("@Resposta:") + 10
                end_resposta = block.find("@", begin_resposta)
                resposta = block[begin_resposta:end_resposta].strip('@\n ')

                parsed.append((pergunta, resposta))
                generated_questions += "@Pergunta:" + pergunta + "\n"
            except:
                print("Bad formatting:", block)
                pass

        print(len(parsed), "perguntas geradas até o momento.")
        print("Ultimas 10 geradas: ", parsed[-10:])

    sintetic_data.extend(parsed[:quantity])

In [ ]:
print(sintetic_data)
print(len(sintetic_data))

###  Salva as perguntas/respostas geradas em um arquivo .csv

In [ ]:
import csv

with open(qa_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Write header if needed
    writer.writerow(['pergunta', 'resposta'])
    # Write the data rows
    writer.writerows(sintetic_data)

print(f'Synthetic data saved to {qa_file_path}')

In [ ]:
# prompt: Download sintetic_data.csv in my computer

from google.colab import files

files.download(qa_file_path)

# Treinando LoRA


### Carrega o dataset
- Usa of formato ### Pergunta: descrição_pergunta\n\n### Resposta: descrição_resposta

In [6]:
df = pd.read_csv(qa_file_path)
df.columns = ['pergunta', 'resposta']

def format_prompt(row):
    return f"### Pergunta:\n{row['pergunta']}\n\n### Resposta:\n{row['resposta']}"

df['text'] = df.apply(format_prompt, axis=1)

dataset = Dataset.from_pandas(df[['text']])

print(f"Dataset carregado com {len(dataset)} QAs.")
print("Exemplo:")
print(dataset[0]['text'])


Dataset carregado com 1000 QAs.
Exemplo:
### Pergunta:
Qual foi o palco da final da Liga dos Campeões da UEFA de 2024-25?

### Resposta:
A final foi disputada na Allianz Arena, em Munique, Alemanha.


In [7]:
# prompt: Divide 10% of the dataset for validation

# Divide the dataset into training and validation sets
train_test_split = dataset.train_test_split(test_size=0.1)
dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Training dataset size: {len(dataset)}")
print(f"Validation dataset size: {len(eval_dataset)}")


Training dataset size: 900
Validation dataset size: 100


### Login no Hugging Face

In [8]:
# prompt: Login on hugging face with HF_TOKEN variables in secret

from huggingface_hub import login
import os
from google.colab import userdata

# Check if HF_TOKEN is in Colab secrets and login
try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(token=hf_token)
        print("Successfully logged in to Hugging Face.")
    else:
        print("HF_TOKEN not found in Colab secrets or is empty.")
except KeyError:
    print("HF_TOKEN not found in Colab secrets.")
except Exception as e:
    print(f"An error occurred during Hugging Face login: {e}")

Successfully logged in to Hugging Face.


### Carrega o modelo base com quantização dupla de 4 bits.

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Carrega o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Adiciona o token de pad para o tokenizer, se não estiver definido
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Carrega o modelo com a configuração de quantização
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    # attn_implementation='eager', # gemma 2 specific
    device_map="auto" # Distribui o modelo automaticamente pela GPU
)

# Prepara o modelo para treinamento em 4 bits (QLoRA)
model = prepare_model_for_kbit_training(model)

print(f"Pegada de memória estimada do modelo: {model.get_memory_footprint() / 1e6:.2f} MB")

Pegada de memória estimada do modelo: 1537.48 MB


### Configura o LoRA

In [10]:
print(model)

print("\nConfigurando o LoRA...")
lora_config = LoraConfig(
    r=16, # Rank da matriz LoRA. Valores comuns: 8, 16, 32. Maiores = mais parâmetros treináveis.
    lora_alpha=32, # Escala os pesos LoRA. Geralmente o dobro de 'r'.
    lora_dropout=0.05, # Dropout para regularização.
    bias="none", # Não treinar bias com LoRA.
    task_type="CAUSAL_LM", # Tipo de tarefa para modelos de linguagem.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # somente nos módulos de atenção
)

for name, param in model.named_parameters():
    if "classifier" in name or "pre_classifier" in name:
        param.requires_grad = True
    elif "lora" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Aplica a configuração LoRA ao modelo
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("LoRA configurado no modelo.")


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

Checa o tamanho das perguntas e respostas pra definir o tamanho do contexto.

In [11]:
import numpy as np

question_lengths = df['pergunta'].str.len()
answer_lengths = df['resposta'].str.len()

percentile_90_question = np.percentile(question_lengths, 90)
percentile_90_answer = np.percentile(answer_lengths, 90)
print(f"Percentil 90 do tamanho da pergunta: {percentile_90_question:.2f}")
print(f"Percentil 90 do tamanho da resposta: {percentile_90_answer:.2f}")

Percentil 90 do tamanho da pergunta: 139.00
Percentil 90 do tamanho da resposta: 203.10


### Configura os parametros de treinamento

In [12]:
training_args = SFTConfig(
    output_dir='./training_output/' + model_id,
    num_train_epochs=5,
    fp16=False,
    bf16=True,
    per_device_train_batch_size=2, # Tamanho do batch por GPU. Reduza se tiver problemas de memória.
    gradient_accumulation_steps=4, # Acumula gradientes para simular um batch maior (2 * 4 = batch efetivo de 8)
    learning_rate=2e-4,
    logging_steps=50,
    # gradient_checkpointing=True, # Economiza memória da GPU (pode desacelerar um pouco o treinamento)
    group_by_length=True, # Otimiza o agrupamento de sequências com tamanhos semelhantes.
    seed=42,
    dataset_text_field="text",
    max_seq_length=512,
    warmup_ratio=0.03, # Proporção de passos para aquecimento da taxa de aprendizado
    lr_scheduler_type="cosine", # Tipo de scheduler da taxa de aprendizado
    report_to="none",

    save_strategy="epoch",
    eval_strategy="epoch",
    per_device_eval_batch_size=2, # Batch size for evaluation (can be larger than train if memory allows)
    load_best_model_at_end=True,  # Load the model with the best validation performance at the end of training
    metric_for_best_model="eval_loss", # Metric to monitor for `load_best_model_at_end`
    greater_is_better=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    peft_config=lora_config,
    args=training_args,
)

Converting train dataset to ChatML:   0%|          | 0/900 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/900 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### Treina e salva o modelo

In [13]:
# Inicia o treinamento
trainer.train()
print("\nTreinamento concluído!")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,1.135800,0.993202
2,0.880100,0.828562
3,0.728900,0.726382
4,0.557400,0.685710
5,0.503900,0.686364


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Treinamento concluído!


In [14]:
trainer.save_model(output_model_dir)

In [15]:
# 1. Especifique o caminho da pasta e o nome do arquivo ZIP de saída
folder_to_zip = output_model_dir # Substitua
output_zip_name = "lora.zip"

# 2. Crie o arquivo ZIP usando o comando zip
!zip -r {output_zip_name} {folder_to_zip}

# 3. Baixe o arquivo ZIP
from google.colab import files
files.download(output_zip_name)

updating: lora_folder/ (stored 0%)
updating: lora_folder/tokenizer.json (deflated 85%)
updating: lora_folder/chat_template.jinja (deflated 71%)
updating: lora_folder/training_args.bin (deflated 52%)
updating: lora_folder/adapter_model.safetensors (deflated 8%)
updating: lora_folder/adapter_config.json (deflated 55%)
updating: lora_folder/tokenizer_config.json (deflated 96%)
updating: lora_folder/special_tokens_map.json (deflated 63%)
updating: lora_folder/README.md (deflated 66%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Inferência e Exemplos

Função que faz a inferência

In [5]:
def fazer_inferencia(pergunta, modelo_ajustado, tokenizer):
    prompt = f"### Pergunta:\n{pergunta}\n\n### Resposta:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = modelo_ajustado.generate(**inputs, max_new_tokens=200, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    start_of_response = response.find("### Resposta:")
    if start_of_response != -1:
        response_text = response[start_of_response + len("### Resposta:"):].strip()
        return response_text
    else:
        return response

Inicializando as questões exemplo.

In [6]:
test_questions = [
    "Quem foi o campeão da Champions League 2024/2025?",
    "Qual foi o vice-campeão da Champions League 2024/2025?",
    "Quem foram os artilheiros da Champions League 2024/2025?",
    "Quais times chegaram às semifinais da Champions League 2024/2025?",
    "Quem eliminou o Real Madrid da Champions League 2024/2025?",
    "Qual foi a maior surpresa da Champions League 2024/2025?",
    "Qual foi a maior decepção da Champions League 2024/2025?",
    "Quais jogadores jovens se destacaram na Champions League 2024/2025 e podem ser as futuras estrelas?",
    "Houve alguma polêmica significativa envolvendo arbitragem ou fair play na Champions League 2024/2025?",
    "Houve alguma mudança significativa nas regras da Champions League 2024/2025?",
]



### Carrega o modelo quantizado sem o LoRA

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Adiciona o token de pad para o tokenizer, se não estiver definido
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Carrega o modelo com a configuração de quantização
model_base = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    # attn_implementation='eager', # gemma 2 specific
    device_map="auto" # Distribui o modelo automaticamente pela GPU
)


print(f"Pegada de memória estimada do modelo: {model_base.get_memory_footprint() / 1e6:.2f} MB")

Pegada de memória estimada do modelo: 1012.01 MB


### Gera exemplos pro modelo sem LoRA

In [8]:
print("### Respostas do modelo base sem LoRA ###")
for i, pergunta in enumerate(test_questions):
    print(f"\n--- Consulta {i+1} ---")
    print(f"Pergunta: {pergunta}")
    resposta = fazer_inferencia(pergunta, model_base, tokenizer)
    print(f"Resposta: {resposta}")

### Respostas do modelo base sem LoRA ###

--- Consulta 1 ---
Pergunta: Quem foi o campeão da Champions League 2024/2025?
Resposta: O campeão da Champions League 2024/2025 foi Real Madrid.

--- Consulta 2 ---
Pergunta: Qual foi o vice-campeão da Champions League 2024/2025?
Resposta: A resposta correta é Cristiano Ronaldo.

--- Consulta 3 ---
Pergunta: Quem foram os artilheiros da Champions League 2024/2025?
Resposta: A resposta correta é o time de Manchester City. 

### Exemplo de resolução:
A resposta correta é o time de Manchester City. 

### Exemplo de explicação:
Os artilheiros da Champions League 2024/2025 foram Gabriel Jesus, João Cancelo, Oleksandr Zinchenko e Kevin De Bruyne.

--- Consulta 4 ---
Pergunta: Quais times chegaram às semifinais da Champions League 2024/2025?
Resposta: Não foi possível determinar os jogos específicos das semifinais da Champions League 2024/2025. No entanto, posso fornecer uma lista com os jogos e equipes que chegaram à semifinal da competição. 

1. M

### Carrega o modelo com LoRA

In [9]:
from peft import PeftModel

model_lora = PeftModel.from_pretrained(model_base, output_model_dir)

print(f"Pegada de memória estimada do modelo com LoRA: {model_lora.get_memory_footprint() / 1e6:.2f} MB")

Pegada de memória estimada do modelo com LoRA: 1025.64 MB


### Gera exemplos pro modelo com LoRA

In [10]:
print("\n### Respostas do modelo com LoRA ###")
for i, pergunta in enumerate(test_questions):
    print(f"\n--- Teste {i+1} ---")
    print(f"Pergunta: {pergunta}")
    resposta = fazer_inferencia(pergunta, model_lora, tokenizer)
    print(f"Resposta: {resposta}")


### Respostas do modelo com LoRA ###

--- Teste 1 ---
Pergunta: Quem foi o campeão da Champions League 2024/2025?
Resposta: O Paris Saint-Germain (PSG) conquistou o título da Champions League 2024/2025, em um desfile épico na Allianz Arena, em Munique, na Alemanha.

--- Teste 2 ---
Pergunta: Qual foi o vice-campeão da Champions League 2024/2025?
Resposta: O Arsenal conquistou o título, vencendo os Paris Saint-Germain por 2 a 1 nos pênaltis.

--- Teste 3 ---
Pergunta: Quem foram os artilheiros da Champions League 2024/2025?
Resposta: Ousmane Dembélé (Barcelona) e Raphinha (Barcelona) foram os artilheiros da Champions League 2024/2025.

--- Teste 4 ---
Pergunta: Quais times chegaram às semifinais da Champions League 2024/2025?
Resposta: Real Madrid, Barcelona, Bayern de Munique, Inter de Milão, PSG, Borussia Dortmund, Arsenal e Aston Villa.

--- Teste 5 ---
Pergunta: Quem eliminou o Real Madrid da Champions League 2024/2025?
Resposta: O Paris Saint-Germain eliminou o Real Madrid da Cham